In [1]:
import pandas as pd 

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [46]:
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
import numpy as np 

In [8]:
import re
import nltk
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import sent_tokenize,word_tokenize

In [3]:
df=pd.read_csv('D:\Brand Sentiment Analysis using NLP\Dataset\Dataset - Train.csv')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [6]:
df.isnull().sum()

tweet_text                                            0
emotion_in_tweet_is_directed_at                       0
is_there_an_emotion_directed_at_a_brand_or_product    0
dtype: int64

In [5]:
df=df.dropna()

In [7]:
df.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product':'emotion',
                   'tweet_text':'text'},inplace=True)

In [9]:
def remove_tags(text):
    remove=re.compile(r'')
    return re.sub(remove, '', text)

df['text'] = df['text'].apply(remove_tags)

In [30]:
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [31]:
df.head()

,text,emotion_in_tweet_is_directed_at,emotion
0,iPhone i have a 3g after 3 hrs tweeting at it ...,iPhone,1
1,iPad or iPhone App know about awesome app that...,iPad or iPhone App,3
2,iPad can not wait for 2 they should sale them ...,iPad,3
3,iPad or iPhone App i hope this festival as cra...,iPad or iPhone App,1
4,Google great stuff on fri marissa mayer tim ma...,Google,3


In [11]:
def remove_splchr(text):
    cp=[]
    text=text.split()
    
    for i in text:
        if i.isalnum():
            cp.append(i)
        
    return " ".join(cp)

df['text']=df['text'].apply(remove_splchr)
    

In [13]:
df['text']=df['text'].str.lower()

In [18]:
df['text']=df['emotion_in_tweet_is_directed_at']+ " " + df['text']

In [19]:
input_data=df['text'].tolist()

In [20]:
input_data

['iPhone i have a 3g after 3 hrs tweeting at it was i need to plugin stations at',
 'iPad or iPhone App know about awesome app that likely appreciate for its giving free ts at',
 'iPad can not wait for 2 they should sale them down at',
 'iPad or iPhone App i hope this festival as crashy as this iphone',
 'Google great stuff on fri marissa mayer tim matt mullenweg',
 'Android is just is around the corner and is only a hop skip and a jump from good time to be an fan',
 'iPad or iPhone App beautifully smart and simple idea rt wrote about our ipad app for',
 'Apple counting down the days to plus strong canadian dollar means stock up on apple gear',
 'Android excited to meet the at so i can show them my sprint galaxy s still running android',
 'Android App find start impromptu parties at with i wait til the android app comes',
 'Android App foursquare ups the just in time for still prefer by best looking android app to',
 'Other Google product or service gotta love this google calendar feat

In [22]:
corpus = []
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

for document in input_data:
    # Tokenization
    words = word_tokenize(document)

    # Remove stop words and perform stemming
    filtered_words = [stemmer.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]

    # Join the processed words to form a document
    processed_document = " ".join(filtered_words)
    corpus.append(processed_document)

In [23]:
corpus

['iphon 3g 3 hr tweet need plugin station',
 'ipad iphon app know awesom app like appreci give free ts',
 'ipad wait 2 sale',
 'ipad iphon app hope festiv crashi iphon',
 'googl great stuff fri marissa mayer tim matt mullenweg',
 'android around corner hop skip jump good time fan',
 'ipad iphon app beauti smart simpl idea rt wrote ipad app',
 'appl count day plu strong canadian dollar mean stock appl gear',
 'android excit meet show sprint galaxi still run android',
 'android app find start impromptu parti wait til android app come',
 'android app foursquar up time still prefer best look android app',
 'googl product servic got ta love googl calendar featur top show case check rt via',
 'ipad iphon app great ipad app',
 'ipad iphon app awesom rad ipad app',
 'iphon notic dst come mani iphon user hour late sxsw come sunday',
 'ipad iphon app ad flight match peopl also download iphon nice',
 'ipad iphon app must rt love review forb sxsw ipad app holler gram',
 'ipad need buy ipad2 austin

In [39]:
labels = df['emotion'].tolist()

In [24]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [47]:
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [61]:
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
class_weights = class_weight.compute_class_weight('balanced',classes= np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))



In [62]:
class_weight_dict

{0: 82.25, 1: 1.5893719806763285, 2: 9.013698630136986, 3: 0.3079082826392138}

In [69]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)

In [70]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=16, input_length=100))
model.add(LSTM(units=64, return_sequences=True))  # First LSTM layer with return_sequences=True
model.add(LSTM(units=32, return_sequences=True))  # Second LSTM layer with return_sequences=True
model.add(LSTM(units=16))  # Third LSTM layer
model.add(Dense(units=4, activation='softmax'))


model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [71]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 16)           80000     
                                                                 
 lstm_11 (LSTM)              (None, 100, 64)           20736     
                                                                 
 lstm_12 (LSTM)              (None, 100, 32)           12416     
                                                                 
 lstm_13 (LSTM)              (None, 16)                3136      
                                                                 
 dense_7 (Dense)             (None, 4)                 68        
                                                                 
Total params: 116356 (454.52 KB)
Trainable params: 116356 (454.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
model.fit(X_train, y_train, epochs=20, validation_split=0.2)


Epoch 1/20


66/66 [==============================] - 14s 109ms/step - loss: 0.7554 - accuracy: 0.7971 - val_loss: 0.5713 - val_accuracy: 0.8254
Epoch 2/20
66/66 [==============================] - 5s 81ms/step - loss: 0.5872 - accuracy: 0.8086 - val_loss: 0.5698 - val_accuracy: 0.8254
Epoch 3/20
66/66 [==============================] - 5s 81ms/step - loss: 0.5837 - accuracy: 0.8086 - val_loss: 0.5640 - val_accuracy: 0.8254
Epoch 4/20
66/66 [==============================] - 5s 81ms/step - loss: 0.5825 - accuracy: 0.8086 - val_loss: 0.5625 - val_accuracy: 0.8254
Epoch 5/20
66/66 [==============================] - 5s 82ms/step - loss: 0.5834 - accuracy: 0.8086 - val_loss: 0.5635 - val_accuracy: 0.8254
Epoch 6/20
66/66 [==============================] - 5s 81ms/step - loss: 0.5818 - accuracy: 0.8086 - val_loss: 0.5629 - val_accuracy: 0.8254
Epoch 7/20
66/66 [==============================] - 5s 80ms/step - loss: 0.5821 - accuracy: 0.8086 - val_loss: 0.5630 - val_accuracy: 0.8254
Epoch 8/20
66/66 [====

In [73]:
new_texts = ["I'm so happy with the brand!", "iPhone i have a 3g after 3 hrs tweeting at it was i need to plugin stations at"]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=100, padding='post', truncating='post')

predictions = model.predict(new_padded_sequences)

print(predictions)
# predicted_labels = [label_mapping_rev[i] for i in tf.argmax(predictions, axis=1).numpy()]

# # Display predicted labels for new texts
# for i, text in enumerate(new_texts):
#     print(f"Text: {text}\nPredicted Sentiment: {predicted_labels[i]}\n")

1/1 [==============================] - 2s 2s/step
[[0.00348169 0.16708241 0.02727663 0.8021593 ]
 [0.00348169 0.16708246 0.02727663 0.8021593 ]]
